In [1]:
import mlflow
import pandas as pd
from matplotlib import pyplot as plt

from hyperopt.pyll import scope
from hyperopt import hp, STATUS_OK, fmin, Trials, tpe

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import xgboost as xgb
from sklearn.pipeline import make_pipeline

from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score)

import warnings
warnings.filterwarnings('ignore')

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow1.db')
mlflow.set_experiment('Customer_Churn_Predictions')

2025/01/15 17:31:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/01/15 17:31:26 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/home/fresh/Documents/Workflow/Customer-retention/notebooks/mlruns/1', creation_time=1736958688074, experiment_id='1', last_update_time=1736958688074, lifecycle_stage='active', name='Customer_Churn_Predictions', tags={}>

In [5]:
path = '../data/churn-data/processed_data/churn.csv'
data = pd.read_csv(path)

data.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,male,0,no,no,45,no,no_phone_service,dsl,yes,no,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,female,0,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


In [6]:
data.isnull().sum()

gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [14]:
data.groupby(["gender"])['churn'].count()

gender
female    3483
male      3549
Name: churn, dtype: int64

In [24]:
data["churn"].value_counts()

churn
0    5163
1    1869
Name: count, dtype: int64

2

In [16]:
data["seniorcitizen"].value_counts()

seniorcitizen
0    5890
1    1142
Name: count, dtype: int64

In [ ]:
data.groupby()

In [ ]:
y = data['churn']
X = data.drop(['churn'], axis=1)
X = X.to_dict(orient="records")


(train_x, test_x, 
    train_y, test_y) = train_test_split(X, y, test_size= 0.3, random_state=1993)

In [4]:
def evaluate_model(y_true, y_pred):

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1score = f1_score(y_true, y_pred)

    out = {"accuracy_score" : accuracy, 
            "precision_score" :precision, 
            "recall_score" : recall, 
            "f1_score" : f1score}
    return out

In [ ]:
# Linear Model
c_values = range(1, 50, 5)
for val in c_values:

    with mlflow.start_run():
        mlflow.set_tag('Developer', 'Godwin')
        mlflow.set_tag('model', 'Logistic Regression')
        mlflow.log_param('C', val)

        lr_pipeline = make_pipeline(DictVectorizer(sparse= False),
                                    LogisticRegression(C =val))
        lr_pipeline.fit(train_x, train_y)

        test_pred = lr_pipeline.predict(test_x)
        test_output_eval = evaluate_model(test_y, test_pred)
        mlflow.log_metrics(test_output_eval)
        # mlflow.sklearn.log_model(lr_pipeline, artifact_path="models_mlflow")

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

In [ ]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag('Developer', 'Godwin')
        mlflow.set_tag('model', 'DecisionTree')
        mlflow.log_params(params)

        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                DecisionTreeClassifier(**params))
        pipeline.fit(train_x, train_y)

        prediction = pipeline.predict(test_x)
        prediction_eval = evaluate_model(test_y, prediction)   
        
        mlflow.log_metrics(prediction_eval)
        # mlflow.sklearn.log_model(pipeline, artifact_path="models_mlflow")
        
    return {"loss": -prediction_eval['f1_score'], 'status': STATUS_OK}

space = {"max_depth": hp.randint("max_depth", 1, 15),
        'min_samples_split': hp.randint("min_samples_split", 2, 15),
        'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
        "criterion": hp.choice("criterion", ["gini", "entropy"]),
        }

best_result = fmin(fn= objective,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=50,
                    trials=Trials()
                    )

In [ ]:
def random_forest_objective(params):

    with mlflow.start_run():
        mlflow.set_tag('Developer', 'Godwin')
        mlflow.set_tag('model', 'RandonForest')
        mlflow.log_params(params)

        pipeline = make_pipeline(DictVectorizer(sparse=False),
                                RandomForestClassifier(**params))
        pipeline.fit(train_x, train_y)

        prediction = pipeline.predict(test_x)
        prediction_eval = evaluate_model(test_y, prediction) 

        mlflow.log_metrics(prediction_eval)
        # mlflow.sklearn.log_model(pipeline, artifact_path="models_mlflow")
        
    return {"loss": -prediction_eval['f1_score'], 'status': STATUS_OK}


space = {"n_estimators": hp.choice("n_estimators", [2,5,10, 20, 30, 50, 100,]),
            'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 5)),
            #'min_samples_split': hp.randint("min_samples_split", 2, 15),
           # 'min_samples_leaf': hp.randint("min_samples_leaf", 1, 15),
            "criterion": hp.choice("criterion", ["gini", "entropy"]),
            }

best_result = fmin(fn=random_forest_objective,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=50,
                    trials=Trials())

In [ ]:
class XGBoostTrainer():
    def __init__(self, params, num_boost_round=1000, early_stopping_rounds=50):
        self.params = params
        self.num_boost_round = num_boost_round
        self.early_stopping_rounds = early_stopping_rounds
        self.booster = None
        self.dict_vectorizer = DictVectorizer(sparse=False)

    def fit(self, X, y):
        # Assuming X, y are your training data and labels
        # Convert the input features to a sparse matrix using DictVectorizer
        
        X_sparse = self.dict_vectorizer.fit_transform(X)

        # Create xgb.DMatrix
        dtrain = xgb.DMatrix(X_sparse, label=y)

        # Train the XGBoost model
        self.booster = xgb.train(self.params,
                                 dtrain=dtrain,
                                 num_boost_round=self.num_boost_round,
                                 early_stopping_rounds=self.early_stopping_rounds,
                                 evals=[(dtrain, 'train')],
                                 verbose_eval=50)
        # mlflow.xgboost.log_model(self.booster, artifact_path='models_mlflow')

        return self

    def predict(self, X):
        # Convert the input features to a sparse matrix using DictVectorizer
        X_sparse = self.dict_vectorizer.transform(X)

        # Create xgb.DMatrix
        dmatrix = xgb.DMatrix(X_sparse)

        # Use the trained model for predictions
        predictions = self.booster.predict(dmatrix)
        return predictions

In [ ]:
def objective(params):

    with mlflow.start_run():

        mlflow.set_tag('Developer', 'Godwin')
        mlflow.set_tag("model", "Xgboost")
        
        mlflow.log_params(params)
        
        booster =  make_pipeline(XGBoostTrainer(params = params))
        
        booster.fit(train_x, train_y)
        prediction = booster.predict(test_x)
        prediction = (prediction >= 0.5).astype('int')
        
        prediction_eval = evaluate_model(test_y, prediction)  
        mlflow.log_metrics(prediction_eval)
        

    return {'loss': -prediction_eval['f1_score'], 'status': STATUS_OK}

search_space = {
            'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 3)),
            'learning_rate': hp.loguniform('learning_rate', -3, 0),
            'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
            'objective': 'binary:logistic',  
            'eval_metric': 'logloss',                                             
            'seed': 42
                }

best_result = fmin(
                fn= objective,
                space=search_space,
                algo=tpe.suggest,
                max_evals=50,
                trials=Trials()
                )

In [ ]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///../../databases/mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.f1_score >0.595",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.f1_score ASC"]
)

In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, F1 Score: {run.data.metrics['f1_score']:.4f}")

In [ ]:
for run in runs:
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="Custormer-churn-models")

In [ ]:
model_name = "Custormer-churn-models"
latest_versions = client.get_latest_versions(name=model_name)
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 5
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

In [ ]:
from datetime import datetime


new_stage = 'Staging'
version = 5
date = datetime.today().date() 

client.update_model_version(
    name=model_name,
    version=version,
    description=f"The model version {version}  was transitioned to {new_stage} on {date}"
)